In [1]:
!pip install jupyter-dash

  Obtaining dependency information for jupyter-dash from https://files.pythonhosted.org/packages/38/62/2c51aeb18c6d765c03c911e4d70f029ccb9de1a991d88cf30e362d2433c9/jupyter_dash-0.4.2-py3-none-any.whl.metadata
  Obtaining dependency information for ansi2html from https://files.pythonhosted.org/packages/42/d7/1bc3433a2406b891182c25a7926abe3eb19af5870d540375ebf7795885b3/ansi2html-1.9.1-py3-none-any.whl.metadata
  Using cached ansi2html-1.9.1-py3-none-any.whl.metadata (3.7 kB)
Using cached ansi2html-1.9.1-py3-none-any.whl (17 kB)


In [25]:
from jupyter_dash import JupyterDash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px
import numpy as np
import pandas as pd

# import dataset by using pandas
df = pd.read_csv("zomato.csv")

# import country iso alpha for plotting geo graphs
country_df = pd.read_excel('Country-Code.xlsx')
country_df['Country Code'] = country_df['Country Code'].astype(int)

# Merging the two dataset
zomato_data = pd.merge(df, country_df, on='Country Code', how='left')

# Deleting the country code column from the zomato dataset
zomato_data.drop(columns='Country Code', axis=1, inplace=True)
zomato_data.drop(index=3513, inplace=True)

# missing value of lon and lat columns into nan 
zomato_data['Longitude'] = zomato_data['Longitude'].replace(0, np.nan)
zomato_data['Latitude'] = zomato_data['Latitude'].replace(0, np.nan)

zomato_data['Latitude'].fillna(method='bfill', inplace=True)
zomato_data['Longitude'].fillna(method='bfill', inplace=True)

country_cuisine = zomato_data.groupby(['Country', 'Cuisines'])['Votes'].sum()
country_cuisine = country_cuisine.reset_index().sort_values(['Country', 'Votes'], ascending=False)
famous_cuisines = country_cuisine.groupby('Country').first().reset_index()

city_cuisine = zomato_data.groupby(['City', 'Cuisines'])['Votes'].sum()
city_cuisine = city_cuisine.reset_index().sort_values(['City', 'Votes'], ascending=False)
city_cuisines = city_cuisine.groupby('City').first().reset_index()

mean_rating = zomato_data.groupby(['Country', 'City'])['Aggregate rating'].mean().reset_index()

sales = zomato_data.groupby(['Country', 'City'])['Average Cost for two'].sum().reset_index()

# creating a dashboard
app = JupyterDash(__name__)


app.layout = html.Div([
    html.Div([
        # this the main header of dashboard displaying the name zomato
        html.H1('Zomato Data Analysis and Visualization',
                style={'color':'#ffffff', 'padding-top':'10px', 'textAlign':'center', 'font-style':'italic',
                       'font-family':'Roboto', 'letter-spacing':'-1px', 'font-weight':'bold', 'fontSize':55, 'word-spacing': '5px'}),
        html.H2('Find the best restaurants, cafés and bars in the world', 
                style={'font-family':'Roboto','fontSize':25, 'textAlign':'center', 'color':'#ffffff', 'margin-top':'-30px'})
              ]),
    html.Div([
        # this 2nd division displaying the num of restaurants in worldwide, country and cities
        html.Div([
            html.P(zomato_data.shape[0], style={'font-family':'Lato','fontSize':35, 'textAlign':'center', 'color':'#444444',
                                                'margin-top':'5px'}),
            html.H3('Restaurants In Worldwide', style={'font-family':'Lato', 'color':'#191919', 'font-weight':'bold',
                                                       'textAlign':'center', 'fontSize':15, 'word-spacing':'5px', 'margin-bottom':'3px', 'margin-top':'-30px'})
                 ], style={'display':'inline-block', 'backgroundColor':'#00ff99', 'width':'15%', 'margin-left':'40px',
                           'height':'10%', 'border-radius':'3px', 'box-shadow':'2px 2px 2px #1f2c56'}),
        html.Div([
            html.P(id='numofcountry', children=0, style={'fontSize':35, 'textAlign':'center', 'color':'#444444', 'margin-top':'5px',
                                                         'font-family':'sans-serif'}),
            html.H3('Restaurants In Country', style={'font-family':'Lato', 'color':'#191919', 'font-weight':'bold',
                                                     'textAlign':'center', 'fontSize':15, 'word-spacing':'5px', 'margin-bottom':'3px', 'margin-top':'-30px'})
                 ], style={'display':'inline-block', 'backgroundColor':'#00ff99', 'width':'15%', 'border-radius':'3px', 'margin-left':'10px',
                          'height':'10%','box-shadow':'2px 2px 2px #1f2c56'}),
        html.Div([
            html.P(id='numofcity', children=0, style={'fontSize':35, 'textAlign':'center','color':'#444444', 'font-family':'Lato', 'margin-top':'5px'}),
            html.H3('Restaurants In City', style={'font-family':'Lato', 'color':'#191919', 'font-weight':'normal',
                                                  'textAlign':'center', 'fontSize':15, 'word-spacing':'5px', 'margin-bottom':'3px', 'margin-top':'-30px'})
                 ], style={'display':'inline-block', 'backgroundColor':'#00ff99', 'width':'15%', 'border-radius':'3px', 'margin-left':'10px',
                           'height':'10%','box-shadow':'2px 2px 2px #1f2c56'}),
        html.Div([
            # this portion containing the dropdown of countries and cities
            html.Div([
                 html.Label('Select Country', style={'font-family':'Lato', 'color':'#191919', 'font-weight':'bold',
                                                     'fontSize':18, 'word-spacing':'5px'}),
                 dcc.Dropdown(id='countries_dropdown',
                              placeholder='Select Country',
                              multi=False,
                              clearable=False,
                              value='India',
                              options=[{'label':c, 'value':c} for c in (country_df['Country'])], style={'width':'250px'})
                      ], style={'padding':6, 'flex':1, 'margin-left':'10px'}),
            html.Div([
                  html.Label('Select City', style={'font-family':'Lato', 'color':'#191919', 'font-weight':'bold',
                                                   'fontSize':18, 'word-spacing':'5px'}),
                  dcc.Dropdown(id='cities_dropdown',
                               placeholder='Select City',
                               multi=False,
                               clearable=False,
                               value='New Delhi',
                               options=[], style={'width':'250px'})
                      ], style={'padding':6, 'flex':1, 'margin':'auto', 'margin-left':'10px'}),
                   ], style={'backgroundColor':'#00ff99', 'width':'45%', 'border-radius':'3px', 'margin-left':'10px',
                            'height':'10%', 'box-shadow':'2px 2px 2px #1f2c56','display':'flex', 'flex-direction': 'row'})
              ], style={'display': 'flex', 'flex-direction': 'row'}),
    html.Div([
        # this the 3rd division of the dashboard which is containing the restaurants locations around the world
        #we are displaying the scatter-mapbox which is users easily find the exact locations of the restaurants 
        html.Div([
            html.H4("Analysis of zomato's restaurants locations", style={'font-family':'Montserrat','fontSize':30, 'textAlign':'center',
                                                                         'font-weight':'bold','padding-top':'10px', 'color':'#444444'}),
            dcc.Markdown('''Zomato operates in over 24 countries around the world,
                            but in this platform we only provide 15 countries and has a vast network 
                            of restaurants listed on this platform. Here are some facts about Zomato's restaurant locations.
                            Zomato operates in several major countries, including India, the United States, the United Kingdom, 
                            Canada, Australia, and the United Arab Emirates. In India, Zomato has a particularly strong presence, 
                            with over 8651 restaurants listed on this platform across more than 43 cities.''', 
                            style={'font-family':'sans-serif', 'color':'#191919', 'font-weight':'bold',
                                   'padding-left':'12px', 'padding-right':'12px', 'textAlign':'left', 'fontSize':15}),
            html.H4('Restaurants Locations',
                     style={'color':'#191919', 'font-family':'Montserrat', 'font-weight':'bold', 'fontSize':20,
                            'word-spacing':'5px', 'padding-left':'10px', 'padding-top':'8px'}),
            dcc.Graph(id='world_map', style={'display':'inline-block', 'width':'98%', 'margin':'auto', 'box-shadow':'3px 3px 3px #AFAFAF',
                                             'margin-top':'-10px', 'margin-left':'12px'})
                 ]),
        html.Div([
            #this is fourth div of the dashboard. in this portion i'm showing some graphs which is basically about the zomato data
            html.Div([
                html.H5('General zomato information about countries and cities', style={'font-family':'Montserrat','fontSize':30, 'textAlign':'center',
                                                    'padding-top':'-40px', 'color':'#444444','font-weight':'bold'}),
                dcc.Markdown('''As an online food delivery and restaurant discovery platform, Zomato collects and 
                                analyzes large amounts of data to improve its services and gain insights into customer 
                                behavior. Here are some ways Zomato uses data analysis. Zomato features a wide range of 
                                restaurants with diverse cuisines. Here are some of the popular cuisines available on 
                                Zomato and  by various countries various citis. And also provide restaurants ratings,
                                price range of cuisines, whether the restaurant has provide online service or not.
                             ''', style={'font-family':'sans-serif', 'color':'#191919', 'font-weight':'bold',
                                         'padding-left':'12px', 'padding-right':'12px', 'textAlign':'left', 'fontSize':15})
                    ]),
            html.Div([
                html.Div([
                    # this portion is about the famous cuisine of countries and cities 
                    html.H5('Famous Cuisine In The Country', style={'font-family':'Roboto', 'color':'#191919', 'font-weight':'bold',
                                                                    'textAlign':'center', 'fontSize':20, 'letter-spacing':'-1px', 'word-spacing':'5px'}),
                    html.P(id='country_cuisine', children=[], style={'font-family':'Roboto', 'color':'#191919', 'font-weight':'normal',
                                                                     'textAlign':'center', 'fontSize':20, 'letter-spacing':'-1px', 'word-spacing':'5px'})
                          ], style={'display':'inline-block', 'backgroundColor':'#00ff99', 'width':'29%', 'margin-left':'100px', 
                               'margin-top':'20px', 'box-shadow':'3px 3px 3px #AFAFAF'}),
              html.Div([
                  html.H4('Famouse Cuisine In The City', style={'font-family':'sans-serif', 'color':'#191919', 'font-weight':'bold',
                                                                'textAlign':'center', 'fontSize':20, 'letter-spacing':'-1px', 'word-spacing':'5px'}),
                  html.P(id='city_cuisine', children=[], style={'font-family':'sans-serif', 'color':'#191919', 'font-weight':'normal',
                                                                'textAlign':'center', 'fontSize':20, 'letter-spacing':'-1px', 'word-spacing':'5px'})
                        ], style={'display':'inline-block', 'backgroundColor':'#00ff99', 'width':'29%', 'margin-left':'250px', 
                                  'margin-top':'20px', 'box-shadow':'3px 3px 3px #AFAFAF'})    
                      ], style={'display':'flex', 'flex-direction':'row'})
                  ]),
        html.Div([
            html.Div([
                # this graph is about total sales of zomato restaurants by each cities of countries
                dcc.Graph(id='bar_chart', style={'display':'inline-block','width':'593px', 'margin':'auto', 'margin-left':'12px',
                                                 'box-shadow':'3px 3px 3px #AFAFAF', 'margin-top':'20px'})
                     ]),
            html.Div([
                # this chart is about average cost of cuisines and their aggregated ratings
                dcc.Graph(id='bar_chart1', style={'display':'inline-block', 'width':'593px', 'margin':'auto', 'margin-left':'20px',
                                                  'box-shadow':'3px 3px 3px #AFAFAF', 'margin-top':'20px'})
                     ])      
                 ], style={'display': 'flex', 'flex-direction': 'row'}
                 ),
        html.Div([
            html.Div([
                # this chart is known for whether the restaurants having online delivery or not 
                dcc.Graph(id='bar_chart2', style={'display':'inline-block','width':'593px', 'margin':'auto', 'margin-left':'12px',
                                                  'box-shadow':'3px 3px 3px #AFAFAF', 'margin-top':'20px'})
                    ]),
            html.Div([
                # this pie chart is about cumulative restaurants ratings of each cities of country
                dcc.Graph(id='pie_chart', style={'display':'inline-block', 'width':'593px', 'margin':'auto', 'margin-left':'20px',
                                                 'box-shadow':'3px 3px 3px #AFAFAF', 'margin-top':'20px'})
                    ])
                ], style={'display': 'flex', 'flex-direction': 'row'}),
        html.Div(children=[
            html.Div(children='Created By : Santhana Krishnan')],
                     style={'font-family':'sans-serif', 'color':'#191919', 'font-weight':'normal', 'padding-right':'20px', 'padding-top':'10px',
                            'padding-bottom':'10px', 'textAlign':'right', 'fontSize':15, 'letter-spacing':'-1px', 'word-spacing':'5px'})               
             ],style={'display':'block', 'backgroundColor':'#F8F8F8', 'margin-left':'40px', 'margin-right':'60px', 'border-radius':'3px', 
                  'margin-top':'15px', 'box-shadow':'2px 2px 2px #1f2c56'}
            )
      
], style={'backgroundColor':'black'})

@app.callback(Output('cities_dropdown', 'options'),
              Input('countries_dropdown', 'value'))
def get_cities(countries_dropdown):
  df_result = zomato_data[zomato_data['Country'] == countries_dropdown]
  return [{'label':i, 'value':i} for i in df_result['City'].unique()]

@app.callback(Output('country_cuisine', 'children'),
              Input('countries_dropdown', 'value'))
def get_cuisine(countries_dropdown):
  df_result = famous_cuisines[famous_cuisines['Country'] == countries_dropdown]
  return df_result['Cuisines']

@app.callback(Output('city_cuisine', 'children'),
              Input('cities_dropdown', 'value'))
def get_city_cuisine(cities_dropdown):
  df_result = city_cuisines[city_cuisines['City'] == cities_dropdown]
  return df_result['Cuisines']

@app.callback(Output('numofcountry', 'children'),
              Input('countries_dropdown', 'value'))
def get_cuisines(countries_dropdown):
  df_result = zomato_data[zomato_data['Country'] == countries_dropdown]
  return df_result.shape[0]

@app.callback(Output("numofcity", "children"),
              Input("cities_dropdown", "value"))
def get_city_options(cities_dropdown):
    df_result = zomato_data[zomato_data['City'] == cities_dropdown]
    return df_result.shape[0]

@app.callback(Output('world_map', 'figure'),
              Input('countries_dropdown', 'value'))
def get_map(countries_dropdown):
  map = zomato_data[zomato_data['Country'] == countries_dropdown]
  fig = px.scatter_mapbox(map,
                          lat="Latitude", 
                          lon="Longitude", 
                          hover_name="Country", 
                          hover_data=['City'],
                          color_discrete_sequence=['#E23744'],
                          zoom=1.5)
  fig.update_layout(mapbox_style='open-street-map',
                    margin={"r":0,"t":0,"l":0,"b":0},
                    mapbox = {'center': {'lon':13, 'lat':12}})
  fig.update_traces(marker_size=9)

  return fig


@app.callback(Output('bar_chart', 'figure'),
              Input('countries_dropdown', 'value'))
def update_bar(countries_dropdown):
  country_wise_df = sales[sales['Country'] == countries_dropdown]
  fig1 = px.bar(country_wise_df,
                x='City',
                y='Average Cost for two',
                color_discrete_sequence=['#E23744'])
  #fig1.update_layout(plot_bgcolor="#E8EBFB")
  fig1.update_layout(title_text='Total sales by Cities of each Country', 
                     xaxis_title='Cities',
                     yaxis_title='Total sales(respective currency)',
                     template='plotly_white', title_x=0.5)
  return fig1

@app.callback(Output('bar_chart1', 'figure'),
              Input('cities_dropdown', 'value'))
def get_city(cities_dropdown):
  rat_cost = zomato_data[zomato_data['City'] == cities_dropdown]
  fig2 = px.scatter(rat_cost,
                    x='Average Cost for two',
                    y='Aggregate rating',
                    #color='Average Cost for two',
                    #color_continuous_scale=px.colors.sequential.Reds_r,
                    color_discrete_sequence=['#E23744'],
                    hover_data=['Restaurant Name'])
  fig2.update_layout(title_text='Cost vs Aggregate rating per city',
                     xaxis_title='Cost',
                     yaxis_title='Aggregate Rating',
                     template='plotly_white', title_x=0.5)
  return fig2
  
@app.callback(Output('bar_chart2', 'figure'),
              Input('countries_dropdown', 'value'))
def update_grouped_bar_chart(countries_dropdown):
    countries_wise_df = zomato_data[zomato_data['Country'] == countries_dropdown]   
    fig3=px.histogram(countries_wise_df,
                      x='City', 
                      color="Has Online delivery",
                      barmode='group',
                      color_discrete_sequence=['#CB313D', '#E44B56'])
    fig3.update_layout(template="plotly_white")
    fig3.update_layout(title_text='Restraunts having online delivery service', title_x=0.5)
    
    return fig3

@app.callback(Output("pie_chart", "figure"),
              Input("countries_dropdown", "value"))
def get_rating(countries_dropdown):
  rating = mean_rating[mean_rating['Country'] == countries_dropdown]
  fig4 = px.pie(rating,
                names='City',
                values='Aggregate rating',
                color_discrete_sequence=px.colors.sequential.Reds_r, 
                hole=0.6)
  fig4.update_traces(textposition='inside')
  fig4.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
  fig4.update_layout(title_text="Average ratings of Restaurants by cities")

  return fig4



if __name__ == '__main__':
   app.run_server(debug=True, port=8073)

D:\anaconda\Lib\site-packages\dash\dash.py:538: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



Dash app running on http://127.0.0.1:8073/


In [14]:
from notebook import notebookapp
servers = list(notebookapp.list_running_servers())
print(servers)


[{'base_url': '/', 'hostname': 'localhost', 'notebook_dir': 'C:\\Users\\krish', 'password': False, 'pid': 11100, 'port': 8888, 'secure': False, 'sock': '', 'token': '6069c2e85dc8479a8e90cf84df76a167f17e172c12161c87', 'url': 'http://localhost:8888/'}, {'base_url': '/', 'hostname': 'localhost', 'notebook_dir': 'C:\\Users\\krish', 'password': False, 'pid': 13368, 'port': 8888, 'secure': False, 'sock': '', 'token': 'b3bc99632652a1ce0d19b04d6fa4afa7903a17a49b856b6d', 'url': 'http://localhost:8888/'}, {'base_url': '/', 'hostname': 'localhost', 'notebook_dir': 'C:\\Users\\krish', 'password': False, 'pid': 13556, 'port': 8888, 'secure': False, 'sock': '', 'token': 'ffd759282b88e9b31cdee41ca938ea72a95eaabc46e39008', 'url': 'http://localhost:8888/'}, {'base_url': '/', 'hostname': 'localhost', 'notebook_dir': 'C:\\Users\\krish', 'password': False, 'pid': 16500, 'port': 8888, 'secure': False, 'sock': '', 'token': '8b75aa16464ebe2df6bdbb45db0edba1d40a8057f8787617', 'url': 'http://localhost:8888/'},